In [ ]:
%%html
<style>
.container {
  width: 100%;
}
</style>

In [ ]:
%load_ext nb_mypy

In [ ]:
import nbimporter
from Exercise02AI import Exercise02AI, display_path

# Aufgabe 03: Minimax (simple evaluation function)

Die Funktion `get_next_middle_game_move` nimmt das aktuelle Board als Argument und gibt einen super ausgewählten Zug aus der Menge aller gültigen Züge zurück.   
Alle restlichen Funktionen werden von der Basisklasse `ChessAI` vererbt. 

In [ ]:
import chess

class Exercise03AI(Exercise02AI):
    """Chooses middle game moves using minimax algorithm and piece square tables"""
    def __init__(self, player_name: str, search_depth: int = 2) -> None:
        super().__init__(player_name, search_depth)
        # Create pice square table mapping (needs to revert tables) 
        self.PIECE_TO_PST = {
            chess.PAWN:   self.PST_PAWN[::-1],
            chess.KNIGHT: self.PST_KNIGHT[::-1],
            chess.BISHOP: self.PST_BISHOP[::-1],
            chess.ROOK:   self.PST_ROOK[::-1],
            chess.QUEEN:  self.PST_QUEEN[::-1],
            chess.KING:   self.PST_KING[::-1]
        }

    # Define field values as class constants
    PST_PAWN = [
        0,  0,  0,  0,  0,  0,  0,  0,
       50, 50, 50, 50, 50, 50, 50, 50,
       10, 10, 20, 30, 30, 20, 10, 10,
        5,  5, 10, 25, 25, 10,  5,  5,
        0,  0,  0, 20, 20,  0,  0,  0,
        5, -5,-10,  0,  0,-10, -5,  5,
        5, 10, 10,-20,-20, 10, 10,  5,
        0,  0,  0,  0,  0,  0,  0,  0
    ]

    PST_KNIGHT = [
        -50,-40,-30,-30,-30,-30,-40,-50,
        -40,-20,  0,  0,  0,  0,-20,-40,
        -30,  0, 10, 15, 15, 10,  0,-30,
        -30,  5, 15, 20, 20, 15,  5,-30,
        -30,  0, 15, 20, 20, 15,  0,-30,
        -30,  5, 10, 15, 15, 10,  5,-30,
        -40,-20,  0,  5,  5,  0,-20,-40,
        -50,-40,-30,-30,-30,-30,-40,-50
    ]

    PST_BISHOP = [
        -20,-10,-10,-10,-10,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5, 10, 10,  5,  0,-10,
        -10,  5,  5, 10, 10,  5,  5,-10,
        -10,  0, 10, 10, 10, 10,  0,-10,
        -10, 10, 10, 10, 10, 10, 10,-10,
        -10,  5,  0,  0,  0,  0,  5,-10,
        -20,-10,-10,-10,-10,-10,-10,-20
    ]

    PST_ROOK = [
        0,  0,  0,  0,  0,  0,  0,  0,
        5, 10, 10, 10, 10, 10, 10,  5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
       -5,  0,  0,  0,  0,  0,  0, -5,
        0,  0,  0,  5,  5,  0,  0,  0
    ]

    PST_QUEEN = [
        -20,-10,-10, -5, -5,-10,-10,-20,
        -10,  0,  0,  0,  0,  0,  0,-10,
        -10,  0,  5,  5,  5,  5,  0,-10,
         -5,  0,  5,  5,  5,  5,  0, -5,
          0,  0,  5,  5,  5,  5,  0, -5,
        -10,  5,  5,  5,  5,  5,  0,-10,
        -10,  0,  5,  0,  0,  0,  0,-10,
        -20,-10,-10, -5, -5,-10,-10,-20
    ]

    PST_KING = [
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -30,-40,-40,-50,-50,-40,-40,-30,
        -20,-30,-30,-40,-40,-30,-30,-20,
        -10,-20,-20,-20,-20,-20,-20,-10,
         20, 20,  0,  0,  0,  0, 20, 20,
         20, 30, 10,  0,  0, 10, 30, 20
    ]

## Evaluierungsfunktion

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    def calculate_pst_value(self, board: chess.Board, piece: chess.PieceType, color: chess.Color):
        """Calculates the piece-square table values from given color"""  
        if color == chess.WHITE:
            return sum([self.PIECE_TO_PST[piece][i] for i in board.pieces(piece, color)])
        else:
            return sum([-(self.PIECE_TO_PST[piece][chess.square_mirror(i)]) for i in board.pieces(piece, color)])

In [ ]:
class Exercise03AI(Exercise03AI): # type: ignore
    MATERIAL_VALUE_PAWN = 100
    MATERIAL_VALUE_KNIGHT = 320
    MATERIAL_VALUE_BISHOP = 330
    MATERIAL_VALUE_ROOK = 500
    MATERIAL_VALUE_QUEEN = 900

    def evaluate(self, board: chess.Board) -> int:
        """Evaluates a given board. 
        Returns a positive value if white has a better material value than black."""
        # Get if pieces standing well or pieces standing badly by calculating with the Piece-Square Tables
        rating_pawn_white   = self.calculate_pst_value(board, chess.PAWN,   chess.WHITE)
        rating_pawn_black   = self.calculate_pst_value(board, chess.PAWN,   chess.BLACK)
        rating_knight_white = self.calculate_pst_value(board, chess.KNIGHT, chess.WHITE)
        rating_knight_black = self.calculate_pst_value(board, chess.KNIGHT, chess.BLACK)
        rating_bishop_white = self.calculate_pst_value(board, chess.BISHOP, chess.WHITE)
        rating_bishop_black = self.calculate_pst_value(board, chess.BISHOP, chess.BLACK)
        rating_rook_white   = self.calculate_pst_value(board, chess.ROOK,   chess.WHITE)
        rating_rook_black   = self.calculate_pst_value(board, chess.ROOK,   chess.BLACK)
        rating_queen_white  = self.calculate_pst_value(board, chess.QUEEN,  chess.WHITE)
        rating_queen_black  = self.calculate_pst_value(board, chess.QUEEN,  chess.BLACK)
        rating_king_white   = self.calculate_pst_value(board, chess.KING,   chess.WHITE)
        rating_king_black   = self.calculate_pst_value(board, chess.KING,   chess.BLACK)

        rating = [
            rating_pawn_white,   rating_pawn_black, 
            rating_knight_white, rating_knight_black, 
            rating_bishop_white, rating_bishop_black, 
            rating_rook_white,   rating_rook_black, 
            rating_queen_white,  rating_queen_black, 
            rating_king_white,   rating_king_black
        ]
        # Get material value
        material = super().evaluate(board)
        return sum(rating) + material

In [ ]:
board = chess.Board()
board.set_fen("7B/pbpk4/1p6/2n1pp2/6p1/4P3/P1P2PNP/RNq1K1R1 w - - 0 1")
player = Exercise03AI("Testplayer", 3)
print(player.evaluate(board))
board

In [ ]:
DEPTH = 3
board = chess.Board()
board.set_fen("4k3/8/2n5/7K/5q2/2N5/8/2B5 b - - 0 1")
player = Exercise03AI("Testplayer", DEPTH)
#print(player.minimax(board, DEPTH))
move = player.best_move(board)
print(move)
display_path(board, move, DEPTH)